<a href="https://colab.research.google.com/github/NMPau/Project/blob/main/PDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import DivIcon
import warnings
import scipy as sp
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import seaborn as sns
sns.set_style('white')
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [4]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [5]:
cd DA

/content/drive/My Drive/Colab Notebooks/DA


In [6]:
!ls

airlines.csv  flights.csv    L_AIRPORT_ID.csv
airports.csv  L_AIRPORT.csv  PDA.ipynb


In [7]:
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv', low_memory = False)
aircode1 = pd.read_csv('L_AIRPORT.csv')
aircode2 = pd.read_csv('L_AIRPORT_ID.csv')

Проверка дубликатов

In [ ]:
print(f'Duplicates in df: {flights.iloc[:,1:].duplicated().sum()}')

Duplicates in df: 0


Форматирование кодов аэропортов

In [8]:
aircode1 = aircode1.reset_index()
aircode2 = aircode2.reset_index()
aircodes = pd.merge(aircode1,aircode2,on='Description')
aircode_dict = dict(zip(aircodes['Code_y'].astype(str),aircodes['Code_x']))

In [9]:
flights['ORIGIN_AIRPORT'] = flights['ORIGIN_AIRPORT'].values.astype(str)
flights['DESTINATION_AIRPORT'] = flights['DESTINATION_AIRPORT'].values.astype(str)

for i in range(len(flights)):
    if len(flights['ORIGIN_AIRPORT'][i]) != 3:
        to_replace = flights['ORIGIN_AIRPORT'][i]
        value = aircode_dict[flights['ORIGIN_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)
for i in range(len(flights)):
    if len(flights['DESTINATION_AIRPORT'][i]) != 3:
        to_replace = flights['DESTINATION_AIRPORT'][i]
        value = aircode_dict[flights['DESTINATION_AIRPORT'][i]]
        flights = flights.replace(to_replace, value)

In [ ]:
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


изменение формата времени

In [ ]:
import datetime
def format_heure(chaine):
    if pd.isnull(chaine):
        return np.nan
    else:
        if chaine == 2400: chaine = 0
        chaine = "{0:04d}".format(int(chaine))
        heure = datetime.time(int(chaine[0:2]), int(chaine[2:4]))
        return heure

In [ ]:
flights['SCHEDULED_DEPARTURE'] = flights['SCHEDULED_DEPARTURE'].apply(format_heure)
flights['DEPARTURE_TIME'] = flights['DEPARTURE_TIME'].apply(format_heure)
flights['SCHEDULED_ARRIVAL'] = flights['SCHEDULED_ARRIVAL'].apply(format_heure)
flights['ARRIVAL_TIME'] = flights['ARRIVAL_TIME'].apply(format_heure)

In [ ]:
flights.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,00:05:00,23:54:00,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,04:30:00,04:08:00,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,00:10:00,00:02:00,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,07:50:00,07:41:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,00:20:00,00:18:00,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,08:06:00,08:11:00,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,00:20:00,00:15:00,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,08:05:00,07:56:00,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,00:25:00,00:24:00,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,03:20:00,02:59:00,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  object 
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [ ]:
corr = flights.corr(method='pearson')

Для построения модели для прогнозирования будем использовать опоздание по колонке ARRIVAL_DELAY

In [ ]:
corr['ARRIVAL_DELAY']

YEAR                        NaN
MONTH                 -0.036793
DAY                   -0.003097
DAY_OF_WEEK           -0.017027
FLIGHT_NUMBER          0.018419
SCHEDULED_DEPARTURE    0.100220
DEPARTURE_TIME         0.159787
DEPARTURE_DELAY        0.944672
TAXI_OUT               0.227319
WHEELS_OFF             0.155773
SCHEDULED_TIME        -0.030029
ELAPSED_TIME           0.029120
AIR_TIME              -0.007108
DISTANCE              -0.025444
WHEELS_ON              0.058880
TAXI_IN                0.116628
SCHEDULED_ARRIVAL      0.088824
ARRIVAL_TIME           0.049876
ARRIVAL_DELAY          1.000000
DIVERTED                    NaN
CANCELLED                   NaN
AIR_SYSTEM_DELAY       0.247187
SECURITY_DELAY         0.009655
AIRLINE_DELAY          0.609351
LATE_AIRCRAFT_DELAY    0.522013
WEATHER_DELAY          0.264799
Name: ARRIVAL_DELAY, dtype: float64

In [ ]:
flights.columns

Убираем ненужные колонки

In [10]:
flights = flights.drop(['YEAR', 'DAY', 'FLIGHT_NUMBER','SCHEDULED_TIME','DIVERTED', 'CANCELLED', 'TAIL_NUMBER', 'AIR_TIME', 'CANCELLATION_REASON', 'DISTANCE'], axis = 1)

Проверяем, есть ли данные с пропущенными значениями

In [ ]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,CANCELLATION_REASON,5729195,1.544643
1,WEATHER_DELAY,4755640,18.275040
2,AIRLINE_DELAY,4755640,18.275040
3,SECURITY_DELAY,4755640,18.275040
4,AIR_SYSTEM_DELAY,4755640,18.275040
5,LATE_AIRCRAFT_DELAY,4755640,18.275040
6,ARRIVAL_DELAY,105071,98.194371
7,AIR_TIME,105071,98.194371
8,ELAPSED_TIME,105071,98.194371
9,TAXI_IN,92513,98.410178


Удаляем строки с пропущенными значениями (для нужных переменных уровень заполненности начинаеется с 98% это хорошо - не будем заполнять средним значением). По переменным с причинами задержки - пропуски заполним нулевым значением (будем считать, что была иная причина задержки)

Удаляем пустые значения

In [11]:
flights.dropna(subset=['ARRIVAL_DELAY','ELAPSED_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','WHEELS_OFF','TAXI_OUT','DEPARTURE_DELAY','DEPARTURE_TIME'],inplace=True)

In [ ]:
missing_df = flights.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(flights.shape[0]-missing_df['missing values'])/flights.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)

,variable,missing values,filling factor (%)
0,MONTH,0,100.0
1,AIRLINE_DELAY,0,100.0
2,SECURITY_DELAY,0,100.0
3,AIR_SYSTEM_DELAY,0,100.0
4,ARRIVAL_DELAY,0,100.0
5,ARRIVAL_TIME,0,100.0
6,SCHEDULED_ARRIVAL,0,100.0
7,TAXI_IN,0,100.0
8,WHEELS_ON,0,100.0
9,LATE_AIRCRAFT_DELAY,0,100.0


Заполняем пропущенные значения нулем

In [12]:
flights = flights.fillna(0)

In [ ]:
flights.head(5)

,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,ELAPSED_TIME,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,1,4,AS,ANC,SEA,5,2354.0,-11.0,21.0,15.0,194.0,404.0,4.0,430,408.0,-22.0,0.0,0.0,0.0,0.0,0.0
1,1,4,AA,LAX,PBI,10,2.0,-8.0,12.0,14.0,279.0,737.0,4.0,750,741.0,-9.0,0.0,0.0,0.0,0.0,0.0
2,1,4,US,SFO,CLT,20,18.0,-2.0,16.0,34.0,293.0,800.0,11.0,806,811.0,5.0,0.0,0.0,0.0,0.0,0.0
3,1,4,AA,LAX,MIA,20,15.0,-5.0,15.0,30.0,281.0,748.0,8.0,805,756.0,-9.0,0.0,0.0,0.0,0.0,0.0
4,1,4,AS,SEA,ANC,25,24.0,-1.0,11.0,35.0,215.0,254.0,5.0,320,259.0,-21.0,0.0,0.0,0.0,0.0,0.0


Найти аэропорт с минимальной задержкой вылета

In [ ]:
a0 = flights.loc[flights['DEPARTURE_DELAY'] == flights['DEPARTURE_DELAY'].min()][['ORIGIN_AIRPORT']]

In [ ]:
AIRPORT_min_delay = airports.loc[airports['IATA_CODE'] == a0['ORIGIN_AIRPORT'].values[0]][['AIRPORT']]

In [ ]:
AIRPORT_min_delay

,AIRPORT
9,Adak Airport


Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [ ]:
LA = airports.loc[airports['AIRPORT'] == 'Los Angeles International Airport'][['IATA_CODE']]

In [ ]:
b = flights.loc[flights['DESTINATION_AIRPORT'] == LA['IATA_CODE'].values[0]][['AIRLINE', 'ARRIVAL_DELAY']]

In [ ]:
b1 = b.groupby(['AIRLINE']).mean()
b1

,ARRIVAL_DELAY
AIRLINE,
AA,2.122856
AS,-0.525700
B6,-0.022152
DL,1.398882
F9,10.288235
HA,-4.653317
MQ,18.302469
NK,17.138697
OO,8.202029


Под самой пунктуальной будем понимать авиакомпанию, чье среднее значение самое близкое к нулю (то есть время минимально отклоняется от графика)

In [ ]:
myNumber = 0
m = min(b1['ARRIVAL_DELAY'].values, key=lambda x:abs(x-myNumber))

In [ ]:
ar = b1.loc[b1['ARRIVAL_DELAY'] == m]

In [ ]:
PUNCTUAL_AIRLINE = airlines.loc[airlines['IATA_CODE'] == ar.index[0]][['AIRLINE']]

In [ ]:
PUNCTUAL_AIRLINE

,AIRLINE
4,JetBlue Airways


Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [ ]:
TO = flights[['ORIGIN_AIRPORT', 'TAXI_OUT']]
TI = flights[['DESTINATION_AIRPORT', 'TAXI_IN']]
to = TO.groupby(['ORIGIN_AIRPORT']).mean()
ti = TI.groupby(['DESTINATION_AIRPORT']).mean()

In [ ]:
taxi = to.assign(TAXI_IN = ti.TAXI_IN)
taxi = taxi.assign(TAXI_ALL = taxi.TAXI_OUT + taxi.TAXI_IN)

In [ ]:
taxi

,TAXI_OUT,TAXI_IN,TAXI_ALL
ORIGIN_AIRPORT,,,
ABE,13.325223,3.968635,17.293858
ABI,9.182418,4.003309,13.185727
ABQ,11.948608,5.573155,17.521763
ABR,16.502046,4.679945,21.181991
ABY,12.246849,3.429022,15.675871
...,...,...,...
WRG,7.759547,3.704067,11.463615
WYS,11.903846,4.666667,16.570513
XNA,14.605457,5.102742,19.708200


In [ ]:
max_TAXi = taxi.loc[taxi['TAXI_ALL'] == taxi['TAXI_ALL'].max()]

In [ ]:
max_TAXi_AIRPORT = airports.loc[airports['IATA_CODE'] == max_TAXi.index[0]][['AIRPORT']]

In [ ]:
max_TAXi_AIRPORT

,AIRPORT
182,LaGuardia Airport (Marine Air Terminal)


Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

In [ ]:
!pip install -U lightautoml

In [14]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import f1_score, log_loss, roc_auc_score, roc_curve, precision_score, recall_score, precision_recall_curve
from sklearn.model_selection import train_test_split
import time
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

In [15]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 3 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 300 # Time in seconds for automl run
TARGET_NAME = 'ARRIVAL_DELAY' # Target column name

In [16]:
import torch
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [17]:
Origin_Airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
data = flights[flights['ORIGIN_AIRPORT'] == Origin_Airport]

In [18]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE,  
                                         random_state=RANDOM_STATE)
print('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

Data splitted. Parts sizes: train_data = (17096, 21), test_data = (4274, 21)
CPU times: user 14.8 ms, sys: 54 µs, total: 14.8 ms
Wall time: 17.9 ms


In [19]:
train_data.head()

,MONTH,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,ELAPSED_TIME,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
4425214,10,6,EV,CVG,EWR,1232,1300.0,28.0,56.0,1356.0,162.0,1537.0,5.0,1429,1542.0,73.0,62.0,0.0,11.0,0.0,0.0
1749688,4,3,MQ,CVG,ORD,1225,1219.0,-6.0,11.0,1230.0,69.0,1223.0,5.0,1246,1228.0,-18.0,0.0,0.0,0.0,0.0,0.0
2379402,5,7,OO,CVG,DEN,1541,1622.0,41.0,20.0,1642.0,178.0,1716.0,4.0,1645,1720.0,35.0,0.0,0.0,0.0,35.0,0.0
2965939,7,7,OO,CVG,ORD,2006,2005.0,-1.0,9.0,2014.0,58.0,1957.0,6.0,2024,2003.0,-21.0,0.0,0.0,0.0,0.0,0.0
4601910,10,3,OO,CVG,MEM,1600,1558.0,-2.0,8.0,1606.0,78.0,1612.0,4.0,1629,1616.0,-13.0,0.0,0.0,0.0,0.0,0.0


In [20]:
task = Task('reg', ) 
            #loss = 'mae', metric = 'mae')

In [22]:
roles = {'target': TARGET_NAME, 'drop' : ['DEPARTURE_DELAY']}

In [23]:
automl = TabularAutoML(
    task = task, 
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE}
)

In [ ]:
%%time 
oof_pred = automl.fit_predict(train_data, roles = roles, verbose = 1)

In [25]:
print(automl.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 0.66821 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.16917 * (1 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.16262 * (3 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) 


In [26]:
%%time

test_pred = automl.predict(test_data)
print(f'Prediction for te_data:\n{test_pred}\nShape = {test_pred.shape}')

Prediction for te_data:
array([[ 46.240314  ],
       [ -2.5413773 ],
       [ -9.121152  ],
       ...,
       [ -6.22356   ],
       [  0.44414097],
       [-15.41166   ]], dtype=float32)
Shape = (4274, 1)
CPU times: user 2.65 s, sys: 11.6 ms, total: 2.66 s
Wall time: 1.55 s


In [27]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt 
print(f'TRAIN out-of-fold score: {mean_absolute_error(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'HOLDOUT score: {mean_absolute_error(test_data[TARGET_NAME].values, test_pred.data[:, 0])}')

TRAIN out-of-fold score: 5.3441692324406915
HOLDOUT score: 4.906139193392966


In [28]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(np.array(test_pred.data[:, 0]), np.array(test_data[TARGET_NAME].values))
print("rms error is: " + str(rmse_val))

rms error is: 7.238457295840593


In [29]:
from sklearn.metrics import r2_score
print(f'R^2 train: {r2_score(train_data[TARGET_NAME].values, oof_pred.data[:, 0])}')
print(f'R^2 test: {r2_score(test_data[TARGET_NAME].values, test_pred.data[:, 0])}')

R^2 train: 0.9720221405423547
R^2 test: 0.9719950800914491


In [30]:
from sklearn.metrics import mean_squared_error
not_nan = np.any(~np.isnan(oof_pred.data), axis=1)
print('Check scores:')
print('OOF score: {}'.format(sqrt(mean_squared_error(train_data[TARGET_NAME].values[not_nan], oof_pred.data[not_nan][:, 0]))))
print('TEST score: {}'.format(sqrt(mean_squared_error(test_data[TARGET_NAME].values,test_pred.data[:, 0]))))

Check scores:
OOF score: 7.3450245846178905
TEST score: 7.238457295840593


In [31]:
td = test_data[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'ARRIVAL_DELAY']]
df = pd.DataFrame(test_pred.data[:, 0], columns = ['Prediction_ARRIVAL_DELAY'])

In [32]:
td.index = np.arange(0,len(td))

In [33]:
a = pd.concat([td,df], sort=False, axis=1)
a.head()

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,ARRIVAL_DELAY,Prediction_ARRIVAL_DELAY
0,CVG,IAH,37.0,46.240314
1,CVG,ATL,-3.0,-2.541377
2,CVG,XNA,-9.0,-9.121152
3,CVG,ATL,-8.0,-5.457493
4,CVG,ORD,4.0,-2.510585


In [ ]:
#a.assign(dif = a.Prediction_ARRIVAL_DELAY - rmse_val)

In [45]:
j = a[a['DESTINATION_AIRPORT'] == 'IAH']

In [46]:
jj = np.sqrt(mean_squared_error(j.ARRIVAL_DELAY, j.Prediction_ARRIVAL_DELAY))

In [47]:
jj

6.945596384882649

In [50]:
len(a['DESTINATION_AIRPORT'].unique())

37

In [43]:
rmse = np.sqrt(mean_squared_error(a.ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == 'IAH'], a.Prediction_ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == 'IAH']))

In [44]:
rmse

6.945596384882649

In [58]:
final = []

In [59]:
for uniq_aiport in a['DESTINATION_AIRPORT'].unique():
  rmse = np.sqrt(mean_squared_error(a.ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == uniq_aiport], a.Prediction_ARRIVAL_DELAY[a['DESTINATION_AIRPORT'] == uniq_aiport]))
  final.append([Origin_Airport, uniq_aiport, rmse])


In [ ]:
final

In [64]:
F = pd.DataFrame(final)
F.columns = ['Arrival_Airport', 'Destination_Airport', 'RMSE']

In [69]:
F = F.sort_values(by=['RMSE'])

In [70]:
F.head(3)

,Arrival_Airport,Destination_Airport,RMSE
27,CVG,MSN,2.992403
22,CVG,MEM,3.902920
12,CVG,DCA,4.357968
